# Cross-Validation of `lm` model

Make a k-fold cross-validation using natural years. _Leave-one-(natural year)-out_

In [1]:
# Load packages
suppressMessages(library(lubridate))
suppressMessages(library(tidyverse))
suppressMessages(library(openair))

suppressMessages(library(caret))
suppressMessages(library(BBmisc))

suppressMessages(library(gridExtra))

In [2]:
suppressMessages(library(repr))
options(repr.plot.width=25,
        repr.plot.height=10,
        #repr.plot.pointsize=50,
        repr.plot.family='serif'
       )

In [3]:
# Working directory
setwd("~/Repositories/AirQualityCOVID")

## Representative sample 

In [4]:
sites.lv <- c("es0118a", "es1438a") # Big cities (Madrid and Barcelona)",
sites.lv <- c(sites.lv, "es1580a", "es1340a") # small cities (Santander and Huelva)

save <- FALSE

## DataFrame Creation

In [5]:
require(tidyverse)

In [8]:
# Create dataSet
load("data/all/data_AQ.rda")
load("data/all/meteorology.rda")

aq <- data_AQ %>%
            filter(site %in% sites.lv,
                   date < lubridate::ymd("2020-01-01")
                  ) %>%
            pivot_wider(names_from = variable, values_from = value) %>%
            openair::timeAverage(avg.time = "day", type=c("site")) %>%
            mutate(date = lubridate::as_date(date))

aq[is.na(aq)] <- NA

mto <- data_Mto %>%
            filter(site %in% sites.lv,
                   date < lubridate::ymd("2020-01-01"))

df <- merge(aq, mto,
              by = c("date", "site"), all.x=T) %>%
        #drop_na() %>%
        mutate_if(is.factor, as.character)

rm(data_AQ)
rm(data_Mto)

In [9]:
head(df)

,date,site,no,no2,o3,pm10,pm2.5,ws,wd,tmed,prec,tmin,tmax,presMax,presMin,RH,solar.radiation
,<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2013-01-01,es0118a,33.125000,41.95833,18.20833,8.833333,10.83333,0.8571429,225.2949,6.4,0.00,3.4,9.4,945.9,940.2,80.65828,58.83178
2,2013-01-01,es1340a,4.708333,26.04167,NA,NA,NA,NA,NA,12.6,0.00,9.1,16.0,1023.5,1017.7,81.26930,113.25568
3,2013-01-01,es1438a,31.208333,50.29167,23.16667,NA,NA,3.7791667,336.1693,7.2,0.05,5.3,9.0,969.5,965.6,77.62205,93.89947
4,2013-01-01,es1580a,3.708333,14.91667,NA,21.125000,NA,8.3500000,286.6196,11.6,6.20,10.8,12.3,1019.9,1008.0,88.11917,78.56742
5,2013-01-02,es0118a,101.000000,59.91667,15.25000,11.125000,13.25000,0.9321429,355.5002,5.8,0.00,0.7,10.8,950.3,945.9,75.60469,71.71485
6,2013-01-02,es1340a,21.500000,26.83333,NA,NA,NA,NA,NA,11.3,0.00,5.1,17.5,1026.0,1023.0,74.36909,104.70722


## PreProcess

In [10]:
deseason <- function(cl) {
    # Remove seasonal component of the additive time serie.
    #     This function is called below by an apply
    
    dcomp <- decompose(ts(cl, frequency = 365))

    cl - dcomp$seasonal
}

range.df <- function(df, omit.cl) {
    # Scale dataframe ignoring no numeric columns (omit.cl)
    
    
    cbind(df[, omit.cl],
    apply(X=df[, -omit.cl], MARGIN = 2, FUN = function(cl) {
              (cl - min(cl))/ (max(cl) - min(cl))
          }))
}

filter.IQR <- function(dat, n, qntl=0.75) {
    
    min.IQ <- quantile(dat$value, 1-qntl)
    max.IQ <- quantile(dat$value, qntl)
    
    interval.IQR <- max.IQ - min.IQ
    
    dat[which(dat$value > (min.IQ - (n*interval.IQR)) &
              dat$value < (max.IQ + (n*interval.IQR))),]
}

## Cross-Validation

In [12]:
leave.one.year.out <- function(yr, dat, omit.cl) { 
    # Do k-iteration of k-fold cv by pop yr data for test 
    #     and the rest for training
    #
    # @params: omit.cl: columns to omit in the regression
    
    test <- which(year(dat$date) == yr)
    train <- which(year(dat$date) != yr)

    if(length(test) == 0 | length(train) == 0) {
        return
    }        

    model <- train(value ~., 
                   data=dat[, -omit.cl], subset=train,
                   method="lm"
                  )

    y.th <- predict(model, newdata=dat[test,])

    y.th.qq <- downscaleR:::eqm(dat[train,]$value, 
                                predict(model), 
                                y.th, 
                                n.quantile=99,
                                precip=FALSE, pr.threshold=0,
                                extrapolation="qwerty"
                               )
    
    if (sum(!is.na(dat$date)) > (365*2)) {
        # if there is enough data (more than 2 years), deseasonalized
        ds.dat <- cbind(dat[, omit.cl],
                     apply(dat[, -omit.cl], 2, deseason))
        ds.y.th <- predict(model, newdata=ds.dat[test,])
        
        ds.y.th.qq <- downscaleR:::eqm(dat[train,]$value, 
                                predict(model), 
                                ds.y.th, 
                                n.quantile=99,
                                precip=FALSE, pr.threshold=0,
                                extrapolation="qwerty"
                               )
        
        cor2 <- cor(ds.y.th, ds.dat[test,]$value)
        cor2.qq <- cor(ds.y.th.qq, ds.dat[test,]$value)
    } else {
        cor2 <- NaN 
        cor2.qq <- NaN
    }
    
    data.frame("bias"=mean(y.th) / mean(dat[test,]$value),
               "var.ratio"=var(y.th) / var(dat[test,]$value),
               "cor1"=cor(y.th, dat[test,]$value),
               "cor2"=cor2,
               "RMSE"=sqrt(mean((y.th - dat[test,]$value)^2)),
               ## q-q Mapping 
               "bias.qq"=mean(y.th.qq) / mean(dat[test,]$value),
               "var.ratio.qq"=var(y.th.qq) / var(dat[test,]$value),
               "cor1.qq"=cor(y.th.qq, dat[test,]$value),
               "cor2.qq"=cor2.qq,
               "RMSE.qq"=sqrt(mean((y.th.qq - dat[test,]$value)^2))
              )
}

In [16]:
head(df)

,date,site,no,no2,o3,pm10,pm2.5,ws,wd,tmed,prec,tmin,tmax,presMax,presMin,RH,solar.radiation
,<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2013-01-01,es0118a,33.125000,41.95833,18.20833,8.833333,10.83333,0.8571429,225.2949,6.4,0.00,3.4,9.4,945.9,940.2,80.65828,58.83178
2,2013-01-01,es1340a,4.708333,26.04167,NA,NA,NA,NA,NA,12.6,0.00,9.1,16.0,1023.5,1017.7,81.26930,113.25568
3,2013-01-01,es1438a,31.208333,50.29167,23.16667,NA,NA,3.7791667,336.1693,7.2,0.05,5.3,9.0,969.5,965.6,77.62205,93.89947
4,2013-01-01,es1580a,3.708333,14.91667,NA,21.125000,NA,8.3500000,286.6196,11.6,6.20,10.8,12.3,1019.9,1008.0,88.11917,78.56742
5,2013-01-02,es0118a,101.000000,59.91667,15.25000,11.125000,13.25000,0.9321429,355.5002,5.8,0.00,0.7,10.8,950.3,945.9,75.60469,71.71485
6,2013-01-02,es1340a,21.500000,26.83333,NA,NA,NA,NA,NA,11.3,0.00,5.1,17.5,1026.0,1023.0,74.36909,104.70722


In [19]:
years <- 2013:2019

no.num.cl <- 1:3 # no numeric columns 
cv.df <- data.frame()

# Create one model for each pair of station-pollutant
for (st in sites.lv) {
    data.st <- df[df$site == st,]
    
    for (pll in levels(as.factor(data.st$variable))) {
        dat <- data.st %>%
                filter(variable == pll) %>%
                #select(-site, -variable) %>%
                filter.IQR(coluns=pll,  n=3)
        
        cv.row <- do.call(rbind, lapply(years, leave.one.year.out,
                                        dat, no.num.cl))

        cv.df <- rbind(cv.df,
                       cbind(data.frame("variable"=rep(pll, nrow(cv.row)),
                                        "site"=rep(st, nrow(cv.row))),
                             cv.row)
                       )
    }
}

In [20]:
head(cv.df)

<0 x 0 matrix>

In [14]:
mean.cv.df <- cv.df %>%
                group_by(variable) %>%
                summarise(bias=mean(bias, na.rm=T),
                          var.ratio=mean(var.ratio, na.rm=T),
                          cor1=mean(cor1, na.rm=T),
                          cor2=mean(cor2, na.rm=T),
                          RMSE=mean(RMSE, na.rm=T),
                          ## q-q Mapping
                          bias.qq=mean(bias.qq, na.rm=T),
                          var.ratio.qq=mean(var.ratio.qq, na.rm=T),
                          cor1.qq=mean(cor1.qq, na.rm=T),
                          cor2.qq=mean(cor2.qq, na.rm=T),
                          RMSE.qq=mean(RMSE.qq, na.rm=T)
                         ) %>%
                print()

ERROR: Error: Must group by variables found in `.data`.
* Column `variable` is not found.


In [ ]:
pivot.cv.df.qq <- cv.df %>%
        pivot_longer(cols = 8:12,
                     names_to = "Error", values_to = "Err.Val")

plt.qq <- ggplot(data=pivot.cv.df.qq, aes(x=Error, y=Err.Val, fill=Error)) +
            geom_boxplot() +
            ggtitle("q-q Mapping") +
            theme(plot.title = element_text(hjust = 0.5)) + 
            theme(legend.position = "none") + 
            facet_wrap(~variable, scale="free")

In [ ]:
pivot.cv.df <- cv.df %>%
        pivot_longer(cols = 3:7, 
                     names_to = "Error", values_to = "Err.Val")

plt <- ggplot(data=pivot.cv.df, aes(x=Error, y=Err.Val, fill=Error)) +
            geom_boxplot() +
            ggtitle("Without q-q Mapping") +
            theme(plot.title = element_text(hjust = 0.5)) + 
            theme(legend.position = "none") + 
            facet_wrap(~variable, scale="free")

In [ ]:
all.plt <- grid.arrange(plt, plt.qq, nrow=2)

# Save Results

In [ ]:
if (save == TRUE) {

    write.csv(cv.df, 
              "data/Cross-validation/lm-qq-mapping.csv",
              row.names=F)

    ggsave(
        "lm-CV-qq-mapping.png",
        plot = all.plt,
        device = "png",
        path = "Plots/cross-validation/lm/",
        width=15,
        height=20
    )
}